<a href="https://colab.research.google.com/github/ajibigad/ML-Playground/blob/main/breast_cancer_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

from google.colab import files, drive
import shutil

if not os.path.ismount('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

destination = "/content/drive/MyDrive/Colab Notebooks/breast-cancer.csv"  # Set the destination path in your Drive

if not os.path.exists(destination):
  uploaded = files.upload()
  filename = list(uploaded.keys())[0]  # Get the uploaded filename
  shutil.move(filename, destination)
  print(f"Uploaded file '{filename}' has been moved to Google Drive at '{destination}'")
else:
  print(f"{destination} already exists")

Google Drive is already mounted.
/content/drive/MyDrive/Colab Notebooks/breast-cancer.csv already exists


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load CSV file into a pandas DataFrame
df = pd.read_csv(destination)

# Convert M -> 1 and B -> 0
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

X = df.drop(['id', 'diagnosis'], axis=1).values
y = df['diagnosis'].values

In [101]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape targets to rank 2 tensor eg. from (455) to (455, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape targets to (455, 1)

# Z-score normalization
mean = X_train_tensor.mean(dim=0, keepdim=True)
std = X_train_tensor.std(dim=0, keepdim=True)

X_train_normalized = (X_train_tensor - mean) / std
X_test_normalized = (X_test_tensor - mean) / std

In [6]:
import pdb
import numpy as np

def sigmoid(x): return 1/(1+torch.exp(-x))

def abs_mean_loss(predictions, targets):
  # // abs mean or mse but we did abs mean loss here
  # // for every prediction, if the target is 1, then difference is 1-pred, else difference is abs(0-pred)
  # // predictions will always be between 0 and 1 due to sigmond function
  return torch.where(targets==1, 1-predictions, predictions).mean()

def mse(preds, targets): return ((preds-targets)**2).mean()

def batch_accuracy(predictions, targets):
  with torch.no_grad():
    correct = (predictions>=0.5) == targets
    return correct.float().mean()

def validate_epoch(model, val_data, val_target):
    accs = [batch_accuracy(model.predict(xb), yb) for xb,yb in zip(val_data, val_target)]
    return round(torch.stack(accs).mean().item(), 4)

class LinearModel():
  def __init__(self, weight, bias, x, y, x_validate, y_validate, loss_fn, metrics):
    self.weight = weight
    self.bias = bias
    self.x = x
    self.y = y
    self.x_validate = x_validate
    self.y_validate = y_validate
    self.params = self.weight, self.bias
    self.loss_fn = loss_fn
    self.metrics = metrics
    self.z = None

  def predict(self, x):
    # z(x) = xw + b
    # y = g(z(x)) where g is an activation function
    self.z = x@self.weight + self.bias

    if torch.isnan(self.z).any():
      pdb.set_trace()

    # apply activation function
    return sigmoid(self.z)

  def step(self, loss, learning_rate):
    # get loss
    # calculate gradient, loss.backward()
    # update parameters using learning rate
    loss.backward()
    for p in self.params:
      p.data -= p.grad.data * learning_rate
      p.grad.zero_()

  def train(self, epoch, learning_rate):
    for i in range(epoch):
      preds = self.predict(self.x)
      try:
        loss = self.loss_fn(preds, self.y)
      except:
        print(preds.view(-1).tolist(), self.z)
        raise
      accuracy = self.metrics(preds, self.y)
      validation = validate_epoch(self, self.x_validate, self.y_validate)
      print(f"epoch {i}: loss: {loss}, accuracy: {accuracy}, validation: {validation}")
      self.step(loss, learning_rate)

def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [108]:
w = init_params((X_train_normalized.shape[1], 1)) # size number of
b = init_params(1)

model = LinearModel(w, b, X_train_normalized, y_train_tensor, X_test_normalized, y_test_tensor, nn.BCELoss(), batch_accuracy)
model.train(epoch=50, learning_rate=1e-1)

epoch 0: loss: 1.5447770357131958, accuracy: 0.4571428596973419, validation: 0.4123
epoch 1: loss: 1.3118507862091064, accuracy: 0.5208791494369507, validation: 0.4737
epoch 2: loss: 1.135445475578308, accuracy: 0.5824176073074341, validation: 0.5088
epoch 3: loss: 1.0001778602600098, accuracy: 0.6241758465766907, validation: 0.5702
epoch 4: loss: 0.8958992958068848, accuracy: 0.6747252941131592, validation: 0.614
epoch 5: loss: 0.8134739398956299, accuracy: 0.6989011168479919, validation: 0.6579
epoch 6: loss: 0.7465646862983704, accuracy: 0.7340659499168396, validation: 0.6754
epoch 7: loss: 0.691271185874939, accuracy: 0.7670329809188843, validation: 0.7105
epoch 8: loss: 0.6446295380592346, accuracy: 0.7758241891860962, validation: 0.7281
epoch 9: loss: 0.6045067310333252, accuracy: 0.7868131995201111, validation: 0.7368
epoch 10: loss: 0.5694584250450134, accuracy: 0.795604407787323, validation: 0.7544
epoch 11: loss: 0.5384765863418579, accuracy: 0.8065934181213379, validation: 0

In [8]:
def random_test(model):
  test_X = X[35:67]
  test_y = y[35:67]

  test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
  test_X_tensor_normalized = (test_X_tensor - mean) / std

  # print(test_X)

  preds = (model.predict(test_X_tensor_normalized).view(-1) >= 0.5).tolist()
  targets = test_y >= 0.5
  for pred, target in zip(preds, targets):
    print(f"pred: {pred:>1}, target {target:>1},   match: {pred==target:>1}")

In [9]:
random_test(model)
model.params

pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1


(tensor([[ 0.5258],
         [ 0.8653],
         [ 0.1862],
         [ 0.3394],
         [ 1.4118],
         [ 0.6473],
         [-0.1412],
         [ 0.5237],
         [-1.2163],
         [ 0.2542],
         [-0.5325],
         [ 0.9348],
         [ 2.0133],
         [-0.1452],
         [ 0.7536],
         [-1.1988],
         [-0.8155],
         [ 0.1698],
         [ 0.7537],
         [-1.3546],
         [ 0.0174],
         [-1.3198],
         [ 3.0065],
         [ 0.3005],
         [-0.5575],
         [ 0.8816],
         [-0.5776],
         [ 1.1805],
         [ 0.1884],
         [ 1.0792]], requires_grad=True),
 tensor([0.2826], requires_grad=True))

In [16]:
class NeuralNet(LinearModel):
  def __init__(self, layers, x, y, x_validate, y_validate, loss_fn, metrics):
    self.layers = layers
    self.x = x
    self.y = y
    self.x_validate = x_validate
    self.y_validate = y_validate
    self.loss_fn = loss_fn
    self.metrics = metrics
    self.z = None

  def predict(self, x):
    input = x
    for layer in self.layers[:-1]:
      res = input@layer[0] + layer[1]
      res = res.max(torch.tensor(0.0))
      input = res
      if torch.isnan(res).any():
        print("nan after during prediction in inner layers")
        # pdb.set_trace()

    res = res@self.layers[-1][0] + self.layers[-1][1]
    self.z = res
    if torch.isnan(self.z).any():
      print("nan after during prediction")
      # pdb.set_trace()
    return sigmoid(res)

  def step(self, loss, learning_rate):
    loss.backward()
    for layer in self.layers:
      for p in layer:
        p.data -= p.grad.data * learning_rate
        if torch.isnan(p).any():
          print("nan after backpropagation")
          # pdb.set_trace()
        p.grad.zero_()


In [106]:
neurons = 4
# w1 = init_params((30,neurons))
# b1 = init_params(neurons)
# w2 = init_params((neurons,1))
# b2 = init_params(1)

layer1 = [init_params((30,neurons)), init_params(neurons)]
layer2 = [init_params((neurons, neurons)), init_params(neurons)]
layer3 = [init_params((neurons,1)), init_params(1)]
layers = [layer1, layer2, layer3]

model2 = NeuralNet(layers, X_train_normalized, y_train_tensor, X_test_normalized, y_test_tensor, nn.BCELoss(), batch_accuracy)
model2.train(epoch=50, learning_rate=1e-1)

epoch 0: loss: 1.568037986755371, accuracy: 0.39340659976005554, validation: 0.4211
epoch 1: loss: 1.2484278678894043, accuracy: 0.4087912142276764, validation: 0.4298
epoch 2: loss: 1.0105136632919312, accuracy: 0.44175824522972107, validation: 0.4474
epoch 3: loss: 0.844948947429657, accuracy: 0.47472527623176575, validation: 0.5088
epoch 4: loss: 0.7345471978187561, accuracy: 0.5142857432365417, validation: 0.5263
epoch 5: loss: 0.6667817234992981, accuracy: 0.5450549721717834, validation: 0.5614
epoch 6: loss: 0.6247576475143433, accuracy: 0.5692307949066162, validation: 0.5877
epoch 7: loss: 0.5914564728736877, accuracy: 0.591208815574646, validation: 0.6053
epoch 8: loss: 0.5599461197853088, accuracy: 0.6219780445098877, validation: 0.6228
epoch 9: loss: 0.5338020324707031, accuracy: 0.6461538672447205, validation: 0.6316
epoch 10: loss: 0.5107117295265198, accuracy: 0.6703296899795532, validation: 0.6579
epoch 11: loss: 0.49110299348831177, accuracy: 0.6747252941131592, validati

In [107]:
random_test(model2)
model2.layers

pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 1,   match: 0
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1
pred: 1, target 1,   match: 1
pred: 1, target 1,   match: 1
pred: 0, target 0,   match: 1


[[tensor([[-7.8450e-01, -1.7556e+00, -4.9966e-01, -9.9308e-01],
          [ 2.2695e-01,  8.2368e-01, -1.2140e+00, -1.4660e+00],
          [-1.0965e+00, -8.3788e-01, -1.7847e+00, -2.8014e-01],
          [ 4.3346e-01, -4.5888e-02, -1.3990e-01, -2.3110e-01],
          [-1.1298e+00, -2.5154e-01,  3.3557e-01, -1.5350e-01],
          [ 8.4996e-01,  1.3003e+00, -2.0343e+00, -8.5103e-02],
          [-3.7255e-01,  1.8370e+00,  2.0670e-01, -1.8829e+00],
          [ 4.8928e-01, -9.9342e-01, -8.1885e-01, -3.1312e-01],
          [-1.1011e+00, -1.5873e-01, -9.9768e-01,  9.5884e-02],
          [ 2.5089e-01, -1.0837e+00,  2.0772e-01, -1.5382e-01],
          [ 9.9671e-03, -1.1071e+00, -7.1093e-02, -3.7733e-01],
          [ 2.5278e-01,  9.1610e-01, -7.7673e-01,  3.7842e-02],
          [ 1.4408e+00, -1.0280e+00,  1.2958e+00, -2.0873e+00],
          [ 5.9357e-01,  8.3641e-01, -1.6879e+00,  1.1567e+00],
          [ 9.4891e-01,  1.7472e+00, -1.0126e+00,  4.1104e-01],
          [-1.7691e+00,  1.3614e-01,  1.